<a href="https://colab.research.google.com/github/viniventur/previsao_artigo_HW_SARIMA/blob/main/testes_robustez_holt_winters_artigo_hw_sarima_anpec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Estudo de previsão de receitas utilizando suavização exponencial para os ICMS e IPVA
## Combinações: Aditivo e Multiplicativo e com tendência amortecida (damped trend)

## Baixando pacotes

In [2]:
# Pacotes de gerenciamento do sistema operacional, como manipulação de arquivos e diretórios, gerenciamento de processos, obtenção de informações do sistema, etc.
import os
from google.colab import drive
drive.mount('/content/drive')

# Pacotes para manipulação e tratamento de dados
import numpy as np
import pandas as pd
from datetime import datetime

# Pacotes para análise gráfica
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.dates import YearLocator
from matplotlib.font_manager import FontProperties
import matplotlib as mpl
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
from matplotlib import font_manager
import plotly.graph_objects as go
import plotly.express as px
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

# Pacotes para análise econométrica
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LinearRegression
import warnings
warnings.filterwarnings('ignore')

# ES e Holt-Winters
from statsmodels.tsa.api import SimpleExpSmoothing,Holt,ExponentialSmoothing
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_percentage_error as MAPE
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.seasonal import STL

Mounted at /content/drive


In [3]:
def previsao_hw(nome , df_completo, inicial_treino, limite_treino, periodos_season, numero_previsao):

  '''
  A função a seguir roda todos os 4 modelos (add e mult na sazonalidade e tendência) para o dataframe fornecido de acordo com os parâmetros abaixo:

  Argumentos:

  1. nome (str): Nome da previsão
  2. df_completo (var): Dataframe com todos os dados
  3. inicial_treino (str ou datatime): Data inicial dos dados de treino no formato YYYY-MM-DD
  4. limite_treino (str ou datatime): Data limite dos dados de treino no formato YYYY-MM-DD
  5. periodos_season (int): Períodos referentes a sazonalidade
  6. numero_previsao (int): Número de observações a serem previstas

  Retorno:

  Lista com os nomes das variáveis criadas para a previsão de cada modelo específico e as variáveis dos modelos individualmente contendo os dados concatenados (treino e previsto).
  '''

  # Ajustando a base de treino e real
  df_treino = df_completo.loc[(df_completo.index >= inicial_treino) & (df_completo.index <= limite_treino)]
  df_real = df_completo.loc[(df_completo.index > limite_treino)]
  df_prev_1 = pd.DataFrame(df_treino) # transformando em df caso apenas uma serie seja retornada

  # Realizando a previsão

  # Para realizar previsão de diferentes modelos dos que já estão listados aqui adicione mais itens no dict
  # Ordem: Tendência, Sazonalidade, Damped (bool)

  modelos_dic = {'prev_aa': ['add', 'add', False],
                 'prev_am': ['add', 'mul', False],
                 'prev_mm': ['mul', 'mul', False],
                 'prev_ma': ['mul', 'add', False],
                 'prev_aa_damp': ['add', 'add', True],
                 'prev_am_damp': ['add', 'mul', True],
                 'prev_mm_damp': ['mul', 'mul', True],
                 'prev_ma_damp': ['mul', 'add', True]
                 }

  for i in df_prev_1.columns:

    for nome_df, modelos in modelos_dic.items():

      modelo = ExponentialSmoothing(
                                df_prev_1[i],
                                seasonal_periods=periodos_season,
                                trend=modelos[0],
                                seasonal=modelos[1],
                                damped_trend=modelos[2]
                                ).fit()
      globals()[nome + "_mod_" + nome_df] = modelo
      previsao = modelo.forecast(numero_previsao)
      df_prev_2 = pd.concat([df_treino, previsao], axis=0)
      df_prev_2 = pd.DataFrame(df_prev_2, columns=df_prev_1.columns)
      globals()[nome + "_df_" + nome_df] = df_prev_2

  global lista_modelos
  global lista_df_prev
  lista_modelos = []
  lista_df_prev = []
  for i in modelos_dic.keys():
      lista_modelos.append(nome + "_mod_" + i)
      lista_df_prev.append(nome + "_df_" + i)

  def nomes(modelos_dic):
    print('Variaveis criadas: \n')
    print('Modelos: \n')
    print("lista_modelos \n")
    for i in modelos_dic.keys():
      print(nome + "_mod_" + i)
    print("\nDataframes com previsão: \n")
    print("lista_df_prev \n")
    for i in modelos_dic.keys():
      print(nome + "_df_" + i)

  return

## Tratando dados

In [4]:
df = pd.read_excel("/content/drive/MyDrive/Gerência de Estudos e Projeções/Dados/Impostos/Impostos - ICMS - ITCD - IPVA - IRRF - 1999 - 2024.xlsx", sheet_name='dados_1999_2024')
#df = pd.read_excel("/content/drive/MyDrive/Gerência de Estudos e Projeções/Dados/Impostos/Bases de impostos antigas/bases_2008_2024 - VERSAO ULTRA MEGA MASTER FINAL.xlsx")
df['Total'] = df.ICMS + df.IPVA + df.IRRF + df.ITCD
df

,ANO,MES,ICMS,IPVA,ITCD,IRRF,Total
0,1999,1,4.035843e+07,658956.65,21163.97,3.293230e+03,4.104184e+07
1,1999,2,3.894808e+07,1032605.36,13089.30,8.150040e+03,4.000192e+07
2,1999,3,3.194334e+07,2489990.52,26279.08,0.000000e+00,3.445961e+07
3,1999,4,3.415198e+07,1851256.22,42705.30,0.000000e+00,3.604594e+07
4,1999,5,3.282836e+07,1905139.44,27892.99,0.000000e+00,3.476139e+07
...,...,...,...,...,...,...,...
302,2024,3,6.412978e+08,46837316.37,3990607.28,7.723583e+07,7.693616e+08
303,2024,4,6.901577e+08,64609203.08,4789282.70,8.712705e+07,8.466832e+08
304,2024,5,6.356771e+08,73725244.53,4528985.51,9.139155e+07,8.053229e+08
305,2024,6,6.307851e+08,71891371.15,5737256.86,1.084689e+08,8.168826e+08


In [5]:
# Limpando a base

df.dropna(inplace=True)
df['Data'] = pd.to_datetime(df['ANO'].astype(str) + '-' + df['MES'].astype(str))
df = df.set_index("Data")
df = df.drop(['MES', 'ANO'], axis=1)
df = df.loc[(df.index >= '2008-01-01')] # inicio definido em 01-2008 e fim definido em 12-2023
df

,ICMS,IPVA,ITCD,IRRF,Total
Data,,,,,
2008-01-01,1.435041e+08,3089289.71,272955.72,4.840761e+05,1.473504e+08
2008-02-01,1.339496e+08,4518615.04,110671.20,8.355140e+06,1.469340e+08
2008-03-01,1.263094e+08,6134471.17,215239.97,1.906390e+07,1.517230e+08
2008-04-01,1.245758e+08,8201420.84,207046.89,1.104372e+07,1.440280e+08
2008-05-01,1.288502e+08,8670808.35,239724.30,1.093729e+07,1.486980e+08
...,...,...,...,...,...
2024-02-01,6.385334e+08,40180682.59,5812958.30,8.310542e+07,7.676325e+08
2024-03-01,6.412978e+08,46837316.37,3990607.28,7.723583e+07,7.693616e+08
2024-04-01,6.901577e+08,64609203.08,4789282.70,8.712705e+07,8.466832e+08


In [6]:
# Utilizando apenas ICMS e IPVA
df = df[['ICMS', 'IPVA']]
df

,ICMS,IPVA
Data,,
2008-01-01,1.435041e+08,3089289.71
2008-02-01,1.339496e+08,4518615.04
2008-03-01,1.263094e+08,6134471.17
2008-04-01,1.245758e+08,8201420.84
2008-05-01,1.288502e+08,8670808.35
...,...,...
2024-02-01,6.385334e+08,40180682.59
2024-03-01,6.412978e+08,46837316.37
2024-04-01,6.901577e+08,64609203.08


In [7]:
# dividindo para os testes
df_1 = df.copy()
df_2 = df.copy()
df_1.rename(columns={'ICMS': 'ICMS_1', 'IPVA': 'IPVA_1'}, inplace=True)
df_2.rename(columns={'ICMS': 'ICMS_2', 'IPVA': 'IPVA_2'}, inplace=True)

In [8]:
# treino e real
df_treino = df.loc[(df.index >= '2008-01-01') & (df.index <= '2022-08-01')]
df_real = df.loc[(df.index >= '2022-09-01')]

# TESTE LOAs - 2010 A 2023

In [9]:
# Contabilizar a quantidade de vezes que cada modelo teve o melhor MAPE
contagem_melhor_mape_HW = pd.DataFrame(index=['AA','AM', 'MM', 'MA', 'AAD','AMD', 'MMD', 'MAD'], columns=['quantidade_ICMS', 'quantidade_IPVA'])
contagem_melhor_mape_HW[['quantidade_ICMS', 'quantidade_IPVA']] = 0

anos_com_erro = []

for imposto in ['ICMS', 'IPVA']:
  for ano in range(2010, 2023):
    try:
      previsao_hw(imposto, df[imposto], df.index.min(), f'{ano}-08-01', 12, 16)

      data_ini_prev = f'{ano}-09-01'
      data_fim_prev = f'{ano+1}-12-01'
      df_real = df[imposto]
      df_real = df_real.loc[(df_real.index >= data_ini_prev) & (df_real.index <= data_fim_prev)]

      df_error = pd.DataFrame(index=['AA','AM', 'MM', 'MA', 'AAD','AMD', 'MMD', 'MAD'],
                              columns=['MAPE', 'MAE', 'RMSE', 'AIC', 'BIC', 'AICc'])

      for lin, dfs_prev, mods in zip(range(8), lista_df_prev, lista_modelos):
          df_error.iloc[lin, 0] = round(MAPE(df_real,
                                              eval(dfs_prev).loc[(eval(dfs_prev).index >= data_ini_prev) & (eval(dfs_prev).index <= data_fim_prev)])*100,
                                        2)
          df_error.iloc[lin, 1] = round(MAE(df_real,
                                            eval(dfs_prev).loc[(eval(dfs_prev).index >= data_ini_prev) & (eval(dfs_prev).index <= data_fim_prev)]),
                                        2)
          df_error.iloc[lin, 2] = round(MSE(df_real,
                                            eval(dfs_prev).loc[(eval(dfs_prev).index >= data_ini_prev) & (eval(dfs_prev).index <= data_fim_prev)],
                                            squared=False),2)
          df_error.iloc[lin, 3] = round(eval(mods).aic,2)
          df_error.iloc[lin, 4] = round(eval(mods).bic,2)
          df_error.iloc[lin, 5] = round(eval(mods).aicc,2)

      df_error.sort_values(by='MAPE', inplace=True)

      # Contabilizando o modelo com o melhor MAPE
      best_model = df_error.index[0]
      contagem_melhor_mape_HW.loc[best_model, f'quantidade_{imposto}'] += 1

    except Exception as e:
        # Registrando o ano que causou o erro
        anos_com_erro.append((ano, str(e)))
print(contagem_melhor_mape_HW)

# Exibindo os anos que causaram erro
print("Anos com erro:", anos_com_erro)

     quantidade_ICMS  quantidade_IPVA
AA                 4                1
AM                 0                1
MM                 1                3
MA                 4                0
AAD                1                1
AMD                1                2
MMD                2                2
MAD                0                1
Anos com erro: [(2014, "Input contains infinity or a value too large for dtype('float64')."), (2016, 'Input contains NaN.')]


In [38]:
# Adicionar colunas para armazenar o MAPE de cada modelo
melhores_modelos_HW_ano = pd.DataFrame(index=range(2010, 2023), columns=['ICMS', 'IPVA', 'MAPE_ICMS', 'MAPE_IPVA'])

anos_com_erro = []

for imposto in ['ICMS', 'IPVA']:
    for ano in range(2010, 2023):
        try:
            previsao_hw(imposto, df[imposto], df.index.min(), f'{ano}-08-01', 12, 16)

            data_ini_prev = f'{ano}-09-01'
            data_fim_prev = f'{ano+1}-12-01'
            df_real = df[imposto]
            df_real = df_real.loc[(df_real.index >= data_ini_prev) & (df_real.index <= data_fim_prev)]

            df_error = pd.DataFrame(index=['AA','AM', 'MM', 'MA', 'AAD','AMD', 'MMD', 'MAD'],
                                    columns=['MAPE', 'MAE', 'RMSE', 'AIC', 'BIC', 'AICc'])

            for lin, dfs_prev, mods in zip(range(8), lista_df_prev, lista_modelos):
                df_error.iloc[lin, 0] = round(MAPE(df_real,
                                                    eval(dfs_prev).loc[(eval(dfs_prev).index >= data_ini_prev) & (eval(dfs_prev).index <= data_fim_prev)])*100,
                                              2)
                df_error.iloc[lin, 1] = round(MAE(df_real,
                                                  eval(dfs_prev).loc[(eval(dfs_prev).index >= data_ini_prev) & (eval(dfs_prev).index <= data_fim_prev)]),
                                              2)
                df_error.iloc[lin, 2] = round(MSE(df_real,
                                                  eval(dfs_prev).loc[(eval(dfs_prev).index >= data_ini_prev) & (eval(dfs_prev).index <= data_fim_prev)],
                                                  squared=False),2)
                df_error.iloc[lin, 3] = round(eval(mods).aic,2)
                df_error.iloc[lin, 4] = round(eval(mods).bic,2)
                df_error.iloc[lin, 5] = round(eval(mods).aicc,2)

            df_error.sort_values(by='MAPE', inplace=True)

            # Registrando o modelo com o melhor MAPE
            melhor_modelo = df_error.index[0]
            melhor_mape = df_error.iloc[0, 0]

            melhores_modelos_HW_ano.loc[ano, imposto] = melhor_modelo
            if imposto == 'ICMS':
                melhores_modelos_HW_ano.loc[ano, 'MAPE_ICMS'] = melhor_mape
            else:
                melhores_modelos_HW_ano.loc[ano, 'MAPE_IPVA'] = melhor_mape

        except Exception as e:
            # Registrando o ano que causou o erro
            anos_com_erro.append((ano, imposto, str(e)))

print(melhores_modelos_HW_ano)

# Exibindo os anos que causaram erro
print("Anos com erro:", anos_com_erro)


     ICMS IPVA MAPE_ICMS MAPE_IPVA
2010  AAD  AMD     44.69     12.16
2011   MA  MMD      5.57      9.36
2012  MMD  MMD      4.58     11.58
2013   MA   AM      4.79     10.18
2014   AA  NaN      5.23       NaN
2015   MA   MM      7.82     38.49
2016   AA  NaN      5.25       NaN
2017   AA  AMD      4.74     11.53
2018  MMD   AA       5.0      7.38
2019   MM  AAD      10.1     24.45
2020   MA  MAD     17.46     13.81
2021  AMD   MM      5.24     11.84
2022   AA   MM       8.4     10.59
Anos com erro: [(2014, 'IPVA', "Input contains infinity or a value too large for dtype('float64')."), (2016, 'IPVA', 'Found input variables with inconsistent numbers of samples: [16, 0]')]


In [23]:
'''melhores_modelos_HW_ano.to_excel('/content/drive/MyDrive/Gerência de Estudos e Projeções/ARTIGOS/Artigo HOLT-SARIMA/Resultados/Testes Adicionais - robustez/melhores_modelos_HW_ano.xlsx')
contagem_melhor_mape_HW.to_excel('/content/drive/MyDrive/Gerência de Estudos e Projeções/ARTIGOS/Artigo HOLT-SARIMA/Resultados/Testes Adicionais - robustez/contagem_melhor_mape_HW.xlsx')'''

# TESTE 1 (MESMO CORTE TEMPORAL COM 22 OBS PRA FRENTE)

In [12]:
# treino e real
df_treino = df.loc[(df.index >= '2008-01-01') & (df.index <= '2022-08-01')]
df_real = df.loc[(df.index >= '2022-09-01')]

In [13]:
for col in df_1.columns:
  previsao_hw(col, df_1[col], df_1.index.min(), '2022-08-01', 12, 22)

## Avaliando melhor modelo

### ICMS

In [14]:
data_ini_prev = '2022-09-01'
df_real = df_1['ICMS_1']
df_real = df_real.loc[(df_real.index >= data_ini_prev)]

mods_icms_1 = [ICMS_1_mod_prev_aa,
                   ICMS_1_mod_prev_am,
                   ICMS_1_mod_prev_mm,
                   ICMS_1_mod_prev_ma,
                   ICMS_1_mod_prev_aa_damp,
                   ICMS_1_mod_prev_am_damp,
                   ICMS_1_mod_prev_mm_damp,
                   ICMS_1_mod_prev_ma_damp]

dfs_prev_icms_1 = [ICMS_1_df_prev_aa,
               ICMS_1_df_prev_am,
               ICMS_1_df_prev_mm,
               ICMS_1_df_prev_ma,
               ICMS_1_df_prev_aa_damp,
               ICMS_1_df_prev_am_damp,
               ICMS_1_df_prev_mm_damp,
               ICMS_1_df_prev_ma_damp]

df_error = pd.DataFrame(index=['AA','AM', 'MM', 'MA', 'AAD','AMD', 'MMD', 'MAD'],
                        columns=['MAPE', 'MAE', 'RMSE', 'AIC', 'BIC', 'AICc'])

for lin, dfs_prev, mods in zip(range(0,10), dfs_prev_icms_1, mods_icms_1):
  df_error.iloc[lin, 0] = round(MAPE(df_real,
                                     dfs_prev.loc[(dfs_prev.index >= data_ini_prev)])*100,
                                2)
  df_error.iloc[lin, 1] = round(MAE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)]),
                                2)
  df_error.iloc[lin, 2] = round(MSE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)],
                                    squared=False),2)
  df_error.iloc[lin, 3] = round(mods.aic,2)
  df_error.iloc[lin, 4] = round(mods.bic,2)
  df_error.iloc[lin, 5] = round(mods.aicc,2)
df_error.sort_values(by='MAPE', inplace=True)
df_error_icms_1 = df_error
df_error_icms_1

,MAPE,MAE,RMSE,AIC,BIC,AICc
AA,11.4,71306696.9,89760703.01,6046.59,6097.32,6050.95
MA,11.48,71878957.26,90581697.86,6044.35,6095.08,6048.71
MM,11.88,73612916.5,89601866.11,6034.81,6085.54,6039.16
MAD,12.15,76493374.66,97748377.99,6050.02,6103.91,6054.89
AAD,13.03,81885331.89,103735368.24,6049.88,6103.78,6054.76
AMD,13.32,82851822.29,101502896.57,6039.79,6093.68,6044.66
MMD,14.63,91259463.09,112438053.57,6039.45,6093.34,6044.32
AM,15.11,94278405.32,116095380.5,6036.54,6087.27,6040.9


### IPVA

In [15]:
data_ini_prev = '2022-09-01'
df_real = df['IPVA']
df_real = df_real.loc[(df_real.index >= data_ini_prev)]

dfs_prev_ipva_1 = [IPVA_1_df_prev_aa,
              IPVA_1_df_prev_am,
              IPVA_1_df_prev_mm,
              IPVA_1_df_prev_ma,
              IPVA_1_df_prev_aa_damp,
              IPVA_1_df_prev_am_damp,
              IPVA_1_df_prev_mm_damp,
              IPVA_1_df_prev_ma_damp]

mods_ipva_1 = [IPVA_1_mod_prev_aa,
             IPVA_1_mod_prev_am,
             IPVA_1_mod_prev_mm,
             IPVA_1_mod_prev_ma,
             IPVA_1_mod_prev_aa_damp,
             IPVA_1_mod_prev_am_damp,
             IPVA_1_mod_prev_mm_damp,
             IPVA_1_mod_prev_ma_damp]

df_error = pd.DataFrame(index=['AA','AM', 'MM', 'MA', 'AAD','AMD', 'MMD', 'MAD'],
                        columns=['MAPE', 'MAE', 'RMSE', 'AIC', 'BIC', 'AICc'])

for lin, dfs_prev, mods in zip(range(0,10), dfs_prev_ipva_1, mods_ipva_1):
  df_error.iloc[lin, 0] = round(MAPE(df_real,
                                     dfs_prev.loc[(dfs_prev.index >= data_ini_prev)])*100,
                                2)
  df_error.iloc[lin, 1] = round(MAE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)]),
                                2)
  df_error.iloc[lin, 2] = round(MSE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)],
                                    squared=False),2)
  df_error.iloc[lin, 3] = round(mods.aic,2)
  df_error.iloc[lin, 4] = round(mods.bic,2)
  df_error.iloc[lin, 5] = round(mods.aicc,2)
df_error.sort_values(by='MAPE', inplace=True)
df_error_ipva_1 = df_error
df_error_ipva_1

,MAPE,MAE,RMSE,AIC,BIC,AICc
MAD,11.49,6520896.6,8818249.6,5562.4,5616.3,5567.27
AA,11.51,6460248.79,8706467.47,5559.47,5610.2,5563.83
AAD,11.89,6967785.06,9493275.6,5564.84,5618.74,5569.72
MM,12.61,5968428.13,7847800.44,5582.06,5632.79,5586.42
MA,13.28,6008164.93,7584332.97,5556.03,5606.76,5560.39
MMD,19.54,9290077.79,10964442.5,5572.34,5626.24,5577.21
AM,19.63,9317241.1,10994550.35,5567.92,5618.65,5572.28
AMD,22.84,10991619.18,12849478.36,5576.47,5630.37,5581.34


# TESTE 2 (CORTE EM FEV 23 - 16 MESES PRA FRENTE)

In [16]:
for col in df_2.columns:
  previsao_hw(col, df_2[col], df_2.index.min(), '2023-02-01', 12, 16)

## Avaliando melhor modelo

### ICMS

In [17]:
data_ini_prev = '2023-03-01'
df_real = df_2['ICMS_2']
df_real = df_real.loc[(df_real.index >= data_ini_prev)]

mods_icms_2 = [ICMS_2_mod_prev_aa,
                   ICMS_2_mod_prev_am,
                   ICMS_2_mod_prev_mm,
                   ICMS_2_mod_prev_ma,
                   ICMS_2_mod_prev_aa_damp,
                   ICMS_2_mod_prev_am_damp,
                   ICMS_2_mod_prev_mm_damp,
                   ICMS_2_mod_prev_ma_damp]

dfs_prev_icms_2 = [ICMS_2_df_prev_aa,
               ICMS_2_df_prev_am,
               ICMS_2_df_prev_mm,
               ICMS_2_df_prev_ma,
               ICMS_2_df_prev_aa_damp,
               ICMS_2_df_prev_am_damp,
               ICMS_2_df_prev_mm_damp,
               ICMS_2_df_prev_ma_damp]

df_error = pd.DataFrame(index=['AA','AM', 'MM', 'MA', 'AAD','AMD', 'MMD', 'MAD'],
                        columns=['MAPE', 'MAE', 'RMSE', 'AIC', 'BIC', 'AICc'])

for lin, dfs_prev, mods in zip(range(0,10), dfs_prev_icms_2, mods_icms_2):
  df_error.iloc[lin, 0] = round(MAPE(df_real,
                                     dfs_prev.loc[(dfs_prev.index >= data_ini_prev)])*100,
                                2)
  df_error.iloc[lin, 1] = round(MAE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)]),
                                2)
  df_error.iloc[lin, 2] = round(MSE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)],
                                    squared=False),2)
  df_error.iloc[lin, 3] = round(mods.aic,2)
  df_error.iloc[lin, 4] = round(mods.bic,2)
  df_error.iloc[lin, 5] = round(mods.aicc,2)
df_error.sort_values(by='MAPE', inplace=True)
df_error_icms_2 = df_error
df_error_icms_2

,MAPE,MAE,RMSE,AIC,BIC,AICc
MA,16.99,108090254.24,121134744.34,6245.58,6296.84,6249.77
MAD,17.4,110747711.1,124366698.24,6251.05,6305.52,6255.75
AAD,18.35,116701621.79,130606682.42,6250.98,6305.45,6255.67
MM,19.07,119787038.56,129770043.18,6237.61,6288.88,6241.81
AA,19.41,123428382.66,137971659.46,6247.42,6298.69,6251.62
MMD,21.18,133146578.12,144152221.99,6241.09,6295.56,6245.78
AM,21.75,136718663.47,147970077.66,6238.28,6289.55,6242.48
AMD,22.67,142535368.43,154240014.45,6241.76,6296.22,6246.45


### IPVA

In [18]:
data_ini_prev = '2023-03-01'
df_real = df_2['IPVA_2']
df_real = df_real.loc[(df_real.index >= data_ini_prev)]

dfs_prev_ipva_2 = [IPVA_2_df_prev_aa,
              IPVA_2_df_prev_am,
              IPVA_2_df_prev_mm,
              IPVA_2_df_prev_ma,
              IPVA_2_df_prev_aa_damp,
              IPVA_2_df_prev_am_damp,
              IPVA_2_df_prev_mm_damp,
              IPVA_2_df_prev_ma_damp]

mods_ipva_2 = [IPVA_2_mod_prev_aa,
             IPVA_2_mod_prev_am,
             IPVA_2_mod_prev_mm,
             IPVA_2_mod_prev_ma,
             IPVA_2_mod_prev_aa_damp,
             IPVA_2_mod_prev_am_damp,
             IPVA_2_mod_prev_mm_damp,
             IPVA_2_mod_prev_ma_damp]

df_error = pd.DataFrame(index=['AA','AM', 'MM', 'MA', 'AAD','AMD', 'MMD', 'MAD'],
                        columns=['MAPE', 'MAE', 'RMSE', 'AIC', 'BIC', 'AICc'])

for lin, dfs_prev, mods in zip(range(0,10), dfs_prev_ipva_2, mods_ipva_2):
  df_error.iloc[lin, 0] = round(MAPE(df_real,
                                     dfs_prev.loc[(dfs_prev.index >= data_ini_prev)]['IPVA_2'])*100,
                                2)
  df_error.iloc[lin, 1] = round(MAE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)]['IPVA_2']),
                                2)
  df_error.iloc[lin, 2] = round(MSE(df_real,
                                    dfs_prev.loc[(dfs_prev.index >= data_ini_prev)]['IPVA_2'],
                                    squared=False),
                                2)
  df_error.iloc[lin, 3] = round(mods.aic,2)
  df_error.iloc[lin, 4] = round(mods.bic,2)
  df_error.iloc[lin, 5] = round(mods.aicc,2)
df_error.sort_values(by='MAPE', inplace=True)
df_error_ipva_2 = df_error
df_error_ipva_2

,MAPE,MAE,RMSE,AIC,BIC,AICc
AA,9.4,5462414.9,6827188.99,5747.17,5798.44,5751.37
MAD,9.43,5501412.67,6861861.52,5750.16,5804.62,5754.85
AAD,9.98,5663938.09,6907157.72,5752.43,5806.9,5757.12
MA,10.27,5161036.12,6691969.37,5742.86,5794.13,5747.06
AMD,10.46,5446059.46,6643524.85,5769.31,5823.78,5774.01
MM,11.09,6052745.99,8481010.62,5766.77,5818.04,5770.97
MMD,12.53,6728365.43,7809132.58,5764.86,5819.33,5769.55
AM,13.11,7067805.08,8113106.62,5759.15,5810.41,5763.34
